# Estimators

- What is an estimator in scikit learn
    - explain their methods (fit, predict, etc.)

## Linear regression
- Example of linear regression as estimator
- Show how to inspect the output of the estimator

## Logistic regression
- Example of logistic regression as estimator

## SVM
- Example of SVM as estimator

## KMeans
- Example of KMeans as estimator

## How to choose the estimator?
(link to algorithm cheatsheet of sklearn: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

# Performance metrics

- How to choose the performance metrics
- Example of accuracy, roc_auc, confusion matrix